In [ ]:
import vtk
import nibabel as nib
import numpy as np
from nibabel.processing import resample_from_to

Load MRI Volume Data:

In [ ]:
scan_paths = [
    r"Data\Subject_OAS30008_longitudinal_AH\Scan_1\sub-OAS30008_sess-d1327_T1w.nii",
    r"Data\Subject_OAS30008_longitudinal_AH\Scan_2\sub-OAS30008_sess-d3363_run-02_T1w.nii",
    r"Data\Subject_OAS30008_longitudinal_AH\Scan_4\sub-OAS30008_sess-d1313_T1w.nii",
    r"Data\Subject_OAS30008_longitudinal_AH\Scan_5\sub-OAS30008_sess-d2035_T1w.nii"
]

#r"Data\Subject_OAS30008_longitudinal_AH\Scan_3\sub-OAS30008_sess-d0061_run-02_T1w.nii",

volumes = []

for path in scan_paths:
    nii = nib.load(path)
    data = nii.get_fdata().astype(np.float32)
    volumes.append(data)

first_volume = volumes[0]
first_nii = nib.load(scan_paths[0])

vtk_image = vtk.vtkImageData()
vtk_image.SetDimensions(first_volume.shape)
vtk_image.SetSpacing(first_nii.header.get_zooms())
vtk_image.SetOrigin(0, 0, 0)

flat_data = first_volume.flatten(order="F")
vtk_array = vtk.vtkFloatArray()
vtk_array.SetNumberOfValues(len(flat_data))
vtk_array.SetVoidArray(flat_data, len(flat_data), 1)

vtk_image.GetPointData().SetScalars(vtk_array)

       OASISID   OASIS_session_label  days_to_visit  age at visit  WHODIDDX  \
2546  OAS30397  OAS30397_UDSd1_d0000              0         68.95       1.0   
2735  OAS30428  OAS30428_UDSd1_d0000              0         76.01       1.0   
2766  OAS30433  OAS30433_UDSd1_d3725           3725         76.07       1.0   
6101  OAS30930  OAS30930_UDSd1_d0791            791         77.50       1.0   
6140  OAS30935  OAS30935_UDSd1_d4228           4228         80.04       NaN   

      NORMCOG  DEMENTED  MCIAMEM  MCIAPLUS  MCIAPLAN  ...  schizoif  anxiet  \
2546      0.0       1.0      0.0       1.0       NaN  ...       NaN     NaN   
2735      0.0       1.0      0.0       1.0       NaN  ...       NaN     NaN   
2766      0.0       1.0      0.0       1.0       0.0  ...       NaN     NaN   
6101      0.0       1.0      0.0       0.0       NaN  ...       NaN     NaN   
6140      0.0       1.0      0.0       0.0       NaN  ...       NaN     0.0   

      anxietif  delir  delirif  ptsddx  ptsddxif  

FileNotFoundError: No such file or no access: 'Data/Subject_OAS30008_longitudinal_AH/Scan_1/sub-OAS30008_sess-d1327_T1w.nii'

Volume Rendering:

In [3]:
# Mapper
mapper = vtk.vtkGPUVolumeRayCastMapper()
mapper.SetInputData(vtk_image)

# Transfer Functions
opacity_tf = vtk.vtkPiecewiseFunction()
color_tf = vtk.vtkColorTransferFunction()

vmin, vmax = float(first_volume.min()), float(first_volume.max())
p1, p50, p90, p99 = np.percentile(first_volume, [1, 50, 90, 99])
print(vmin, vmax, p1, p50, p90, p99)

opacity_tf.RemoveAllPoints()
opacity_tf.AddPoint(0, 0.0)      # background
opacity_tf.AddPoint(40, 0.05)    # CSF
opacity_tf.AddPoint(80, 0.15)    # gray matter
opacity_tf.AddPoint(120, 0.35)   # white matter

color_tf.RemoveAllPoints()
color_tf.AddRGBPoint(0, 0, 0, 0)
color_tf.AddRGBPoint(40, 0.7, 0.7, 1.0)   # CSF
color_tf.AddRGBPoint(80, 0.9, 0.7, 0.6)   # GM
color_tf.AddRGBPoint(120, 1.0, 1.0, 1.0)  # WM

# Volume Property
volume_prop = vtk.vtkVolumeProperty()
volume_prop.SetScalarOpacity(opacity_tf)
volume_prop.SetColor(color_tf)
volume_prop.ShadeOn()
volume_prop.SetInterpolationTypeToLinear()

# Gradient Opacity
grad_tf = vtk.vtkPiecewiseFunction()
grad_tf.AddPoint(0, 0.0)
grad_tf.AddPoint(20, 0.0)
grad_tf.AddPoint(60, 1.0)

volume_prop.SetGradientOpacity(grad_tf)
volume_prop.SetDisableGradientOpacity(False)

# Volume Actor
volume = vtk.vtkVolume()
volume.SetMapper(mapper)
volume.SetProperty(volume_prop)

# Cutting Plane
plane = vtk.vtkPlane()


plane.SetNormal(0, 1, 0)   # Y direction
plane.SetOrigin(
    first_volume.shape[0] // 2,
    first_volume.shape[1] // 2,
    first_volume.shape[2] // 2
)

mapper.AddClippingPlane(plane)


0.0 1484.0 0.0 8.0 298.0 620.0


Renderer, window, interactor

In [4]:
# Renderer
renderer = vtk.vtkRenderer()
renderer.AddVolume(volume)
renderer.SetBackground(0.1, 0.1, 0.15)

# Render window
render_window = vtk.vtkRenderWindow()
render_window.AddRenderer(renderer)
render_window.SetSize(900, 900)

# Interactor
interactor = vtk.vtkRenderWindowInteractor()
interactor.SetRenderWindow(render_window)

# Interaction style (trackball camera like in labs)
style = vtk.vtkInteractorStyleTrackballCamera()
interactor.SetInteractorStyle(style)

Camera setup

In [5]:
camera = renderer.GetActiveCamera()
camera.SetPosition(0, -1, 0)
camera.SetFocalPoint(0, 0, 0)
camera.SetViewUp(0, 0, 1)
renderer.ResetCamera()

Timer-based animation callback

In [6]:
class LongitudinalAnimator:
    def __init__(self, vtk_image, volumes):
        self.vtk_image = vtk_image
        self.volumes = volumes
        self.index = 0

    def execute(self, obj, event):
        print(f"[TimerEvent] updating frame {self.index} / {len(self.volumes)-1}")

        data = self.volumes[self.index]
        flat = np.asarray(data, dtype=np.float32).ravel(order="F")

        # Update scalars
        vtk_array = vtk.vtkFloatArray()
        vtk_array.SetNumberOfTuples(flat.size)
        vtk_array.SetVoidArray(flat, flat.size, 1)

        self.vtk_image.GetPointData().SetScalars(vtk_array)
        self.vtk_image.Modified()

        # Force a redraw
        obj.GetRenderWindow().Render()

        # Next frame
        self.index = (self.index + 1) % len(self.volumes)


animator = LongitudinalAnimator(vtk_image, volumes)

FreeSurfer

In [7]:
aseg_nii = nib.load(r"Data\Subject_OAS30008_longitudinal_AH\Scan_1\aseg.mgz") 
# Resample aseg into the first MRI's voxel grid + affine 
aseg_resampled_nii = resample_from_to(aseg_nii, (first_nii.shape, first_nii.affine), order=0)
# IMPORTANT: nearest-neighbor for labels ) 
aseg = aseg_resampled_nii.get_fdata().astype(np.int16) 
# IMPORTANT: ensure aseg aligns with your MRI volume 
print("MRI shape:", first_volume.shape, "ASEG shape:", aseg.shape)

MRI shape: (176, 240, 256) ASEG shape: (176, 240, 256)


In [8]:
# LABELS = {
#     "Left-Hippocampus": 17,
#     "Right-Hippocampus": 53,
#     "Left-Lateral-Ventricle": 4,
#     "Right-Lateral-Ventricle": 43,
#     "Left-Cerebral-White-Matter": 2,
#     "Right-Cerebral-White-Matter": 41,
#     "Left-Cerebral-Cortex": 3,
#     "Right-Cerebral-Cortex": 42,
# }

def numpy_to_vtk_image(vol_np, spacing=(1,1,1)):
    vol_np = np.asarray(vol_np)
    vtk_img = vtk.vtkImageData()
    vtk_img.SetDimensions(vol_np.shape)
    vtk_img.SetSpacing(spacing)
    vtk_img.SetOrigin(0, 0, 0)

    flat = vol_np.ravel(order="F")

    if vol_np.dtype == np.uint8:
        vtk_arr = vtk.vtkUnsignedCharArray()
    elif vol_np.dtype == np.int16:
        vtk_arr = vtk.vtkShortArray()
    else:
        vtk_arr = vtk.vtkFloatArray()

    vtk_arr.SetNumberOfTuples(flat.size)
    vtk_arr.SetVoidArray(flat, flat.size, 1)
    vtk_img.GetPointData().SetScalars(vtk_arr)
    return vtk_img

def mask_to_actor(mask_u8, color=(1,0,0), opacity=0.6, spacing=(1,1,1)):
    vtk_mask = numpy_to_vtk_image(mask_u8.astype(np.uint8), spacing=spacing)

    mc = vtk.vtkMarchingCubes()
    mc.SetInputData(vtk_mask)
    mc.SetValue(0, 0.5)  # iso-surface for binary mask
    mc.Update()

    smooth = vtk.vtkSmoothPolyDataFilter()
    smooth.SetInputConnection(mc.GetOutputPort())
    # smooth.SetNumberOfIterations(20)
    # smooth.SetRelaxationFactor(0.1)
    smooth.SetNumberOfIterations(5)
    smooth.SetRelaxationFactor(0.05)

    smooth.Update()

    surf_mapper = vtk.vtkPolyDataMapper()
    surf_mapper.SetInputConnection(smooth.GetOutputPort())
    surf_mapper.ScalarVisibilityOff()


    actor = vtk.vtkActor()
    actor.SetMapper(surf_mapper)
    actor.GetProperty().SetColor(*color)
    actor.GetProperty().SetOpacity(opacity)
    return actor



# If shapes differ, resample aseg to match MRI

spacing = first_nii.header.get_zooms()  # from your MRI NIfTI

def crop_mask(mask, margin=5):
    coords = np.argwhere(mask > 0)
    if coords.size == 0:
        return mask  # nothing to crop
    mins = np.maximum(coords.min(axis=0) - margin, 0)
    maxs = np.minimum(coords.max(axis=0) + margin, np.array(mask.shape) - 1)
    return mask[mins[0]:maxs[0]+1, mins[1]:maxs[1]+1, mins[2]:maxs[2]+1]

hippo_mask = ((aseg == 17) | (aseg == 53)).astype(np.uint8)
vent_mask  = ((aseg == 4)  | (aseg == 43)).astype(np.uint8)

hippo_crop = crop_mask(hippo_mask, margin=6)
vent_crop  = crop_mask(vent_mask,  margin=6)

actors = []
actors.append(mask_to_actor(hippo_crop, color=(1,0,0),   opacity=0.7,  spacing=spacing))
actors.append(mask_to_actor(vent_crop,  color=(0,0.7,1), opacity=0.35, spacing=spacing))


for a in actors:
    renderer.AddActor(a)

GM, WM and Cortex

In [9]:
# def load_freesurfer_surface(path, color=(1, 1, 1), opacity=0.3):
#     # Read FreeSurfer surface (.pial / .white)
#     reader = vtk.vtkFreesurferReader()
#     reader.SetFileName(path)
#     reader.Update()

#     mapper = vtk.vtkPolyDataMapper()
#     mapper.SetInputConnection(reader.GetOutputPort())
#     mapper.ScalarVisibilityOff()

#     actor = vtk.vtkActor()
#     actor.SetMapper(mapper)
#     actor.GetProperty().SetColor(*color)
#     actor.GetProperty().SetOpacity(opacity)

#     return actor

# lh_pial = load_freesurfer_surface(
#     r"Data\Subject_OAS30008_longitudinal_AH\Scan_1\lh.pial",
#     color=(0.85, 0.7, 0.7),
#     opacity=0.35
# )

# rh_pial = load_freesurfer_surface(
#     r"Data\Subject_OAS30008_longitudinal_AH\Scan_1\rh.pial",
#     color=(0.85, 0.7, 0.7),
#     opacity=0.35
# )

# renderer.AddActor(lh_pial)
# renderer.AddActor(rh_pial)

# lh_white = load_freesurfer_surface(
#     r"Data\Subject_OAS30008_longitudinal_AH\Scan_1\lh.white",
#     color=(1.0, 1.0, 1.0),
#     opacity=0.15
# )

# rh_white = load_freesurfer_surface(
#     r"Data\Subject_OAS30008_longitudinal_AH\Scan_1\rh.white",
#     color=(1.0, 1.0, 1.0),
#     opacity=0.15
# )

# renderer.AddActor(lh_white)
# renderer.AddActor(rh_white)



In [10]:
#Render and start interaction

interactor.Initialize()
interactor.AddObserver("TimerEvent", animator.execute)
interactor.CreateRepeatingTimer(1200)
render_window.Render()
interactor.Start()

[TimerEvent] updating frame 0 / 3
[TimerEvent] updating frame 1 / 3
[TimerEvent] updating frame 2 / 3
[TimerEvent] updating frame 3 / 3
[TimerEvent] updating frame 0 / 3
[TimerEvent] updating frame 1 / 3
[TimerEvent] updating frame 2 / 3
[TimerEvent] updating frame 3 / 3
[TimerEvent] updating frame 0 / 3
[TimerEvent] updating frame 1 / 3
[TimerEvent] updating frame 2 / 3
[TimerEvent] updating frame 3 / 3
[TimerEvent] updating frame 0 / 3
[TimerEvent] updating frame 1 / 3
[TimerEvent] updating frame 2 / 3
[TimerEvent] updating frame 3 / 3
[TimerEvent] updating frame 0 / 3
[TimerEvent] updating frame 1 / 3
[TimerEvent] updating frame 2 / 3
[TimerEvent] updating frame 3 / 3
[TimerEvent] updating frame 0 / 3
[TimerEvent] updating frame 1 / 3
[TimerEvent] updating frame 2 / 3
[TimerEvent] updating frame 3 / 3
[TimerEvent] updating frame 0 / 3
[TimerEvent] updating frame 1 / 3
[TimerEvent] updating frame 2 / 3
[TimerEvent] updating frame 3 / 3
[TimerEvent] updating frame 0 / 3
[TimerEvent] u